In [ ]:
# to access functions from root directory
import sys
sys.path.append('/data/ad181/RemoteDir/ada_multigrid_ppo')

In [2]:
%matplotlib notebook
import numpy as np
import time
import pickle
import os
import matplotlib.pyplot as plt
from copy import copy, deepcopy

import gym
from stable_baselines3.ppo import PPO, MlpPolicy
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines3.common.callbacks import CallbackList
from utils.custom_eval_callback import CustomEvalCallback, CustomEvalCallbackParallel
from utils.env_wrappers import StateCoarse, BufferWrapper, EnvCoarseWrapper, StateCoarseMultiGrid
from typing import Callable
from utils.plot_functions import plot_learning
from utils.multigrid_framework_functions import env_wrappers_multigrid, make_env, generate_beta_environement, parallalize_env, multigrid_framework

from model.ressim import Grid
from ressim_env import ResSimEnv_v0, ResSimEnv_v1

/home/ad181/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ad181/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ad181/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ad181/anaconda3/lib/python3.7/site-packages/tensorboard/co

In [3]:
seed=1
case='case_2_multigrid_fixed'
data_dir='./data'
log_dir='./data/'+case

In [4]:
os.makedirs(data_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

In [5]:
with open('../envs_params/env_data/env_train.pkl', 'rb') as input:
    env_train = pickle.load(input)

# define RL model and callbacks

In [6]:
def generate_model(env_train, seed):
    dummy_env =  generate_beta_environement(env_train, 0.5, env_train.p_x, env_train.p_y, seed)
    dummy_env_parallel = parallalize_env(dummy_env, num_actor=64, seed=seed)
    model = PPO(policy=MlpPolicy,
                env=dummy_env_parallel,
                learning_rate = 1e-4,
                n_steps = 40,
                batch_size = 16,
                n_epochs = 20,
                gamma = 0.99,
                gae_lambda = 0.95,
                clip_range = 0.15,
                clip_range_vf = None,
                ent_coef = 0.001,
                vf_coef = 0.5,
                max_grad_norm = 0.5,
                use_sde= False,
                create_eval_env= False,
                policy_kwargs = dict(net_arch=[70,70,50], log_std_init=-1.7),
                verbose = 1,
                target_kl =0.1,
                seed = seed,
                device = "auto")
    return model

def generate_callback(env_train, best_model_save_path, log_path, eval_freq):
    dummy_env = generate_beta_environement(env_train, 0.5, env_train.p_x, env_train.p_y, seed)
    callback = CustomEvalCallbackParallel(dummy_env, 
                                          best_model_save_path=best_model_save_path, 
                                          n_eval_episodes=1,
                                          log_path=log_path, 
                                          eval_freq=eval_freq)
    return callback

# multigrid framework

In [ ]:
for seed in range(1,4):
    model = multigrid_framework(env_train, 
                                generate_model,
                                generate_callback, 
                                delta_pcent=0.2, 
                                n=np.inf,
                                grid_fidelity_factor_array =[0.25, 0.5, 1.0],
                                episode_limit_array=[50000, 50000, 50000], 
                                log_dir=log_dir,
                                seed=seed)

/data/ad181/RemoteDir/ada_multigrid_ppo/utils/coarse_grid_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):


Using cuda device
seed 1: grid fidelity factor 0.25 learning ..
environement grid size (nx x ny ): 7 x 22


/data/ad181/RemoteDir/ada_multigrid_ppo/utils/custom_eval_callback.py:291: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.subproc_vec_env.SubprocVecEnv object at 0x7f645b008dd8> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7f645b008a90>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


Eval num_timesteps=2560, episode_reward=0.68 +/- 0.00
Episode length: 5.00 +/- 0.00
New best mean reward!
---------------------------------
| eval/              |          |
|    mean_ep_length  | 5        |
|    mean_reward     | 0.679    |
| time/              |          |
|    fps             | 137      |
|    iterations      | 1        |
|    time_elapsed    | 18       |
|    total_timesteps | 2560     |
---------------------------------
policy iteration runtime: 50 seconds

Total episode rollouts: 512

Eval num_timesteps=2560, episode_reward=0.68 +/- 0.00
Episode length: 5.00 +/- 0.00
---------------------------------------
| eval/                   |           |
|    mean_ep_length       | 5         |
|    mean_reward          | 0.679     |
| time/                   |           |
|    fps                  | 746       |
|    iterations           | 1         |
|    time_elapsed         | 3         |
|    total_timesteps      | 2560      |
| train/                  |           |
|  

policy iteration runtime: 37 seconds

Total episode rollouts: 4608

Eval num_timesteps=2560, episode_reward=0.72 +/- 0.00
Episode length: 5.00 +/- 0.00
New best mean reward!
--------------------------------------
| eval/                   |          |
|    mean_ep_length       | 5        |
|    mean_reward          | 0.717    |
| time/                   |          |
|    fps                  | 736      |
|    iterations           | 1        |
|    time_elapsed         | 3        |
|    total_timesteps      | 2560     |
| train/                  |          |
|    approx_kl            | 0.049085 |
|    clip_fraction        | 0.493    |
|    clip_range           | 0.15     |
|    entropy_loss         | 6.11     |
|    explained_variance   | 0.961    |
|    learning_rate        | 0.0001   |
|    loss                 | -0.0788  |
|    n_updates            | 180      |
|    policy_gradient_loss | -0.0467  |
|    std                  | 0.181    |
|    value_loss           | 0.00163  |
-------

...

Early stopping at step 14 due to reaching max kl: 0.15
policy iteration runtime: 29 seconds

Total episode rollouts: 137216

Eval num_timesteps=2560, episode_reward=0.85 +/- 0.00
Episode length: 5.00 +/- 0.00
---------------------------------------
| eval/                   |           |
|    mean_ep_length       | 5         |
|    mean_reward          | 0.853     |
| time/                   |           |
|    fps                  | 345       |
|    iterations           | 1         |
|    time_elapsed         | 7         |
|    total_timesteps      | 2560      |
| train/                  |           |
|    approx_kl            | 0.1529886 |
|    clip_fraction        | 0.665     |
|    clip_range           | 0.15      |
|    entropy_loss         | 14.6      |
|    explained_variance   | 0.979     |
|    learning_rate        | 0.0001    |
|    loss                 | 0.096     |
|    n_updates            | 5360      |
|    policy_gradient_loss | -0.00577  |
|    std                  | 0.1

policy iteration runtime: 38 seconds

Total episode rollouts: 141312

Eval num_timesteps=2560, episode_reward=0.85 +/- 0.00
Episode length: 5.00 +/- 0.00
----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 5          |
|    mean_reward          | 0.854      |
| time/                   |            |
|    fps                  | 348        |
|    iterations           | 1          |
|    time_elapsed         | 7          |
|    total_timesteps      | 2560       |
| train/                  |            |
|    approx_kl            | 0.12749545 |
|    clip_fraction        | 0.669      |
|    clip_range           | 0.15       |
|    entropy_loss         | 14.6       |
|    explained_variance   | 0.982      |
|    learning_rate        | 0.0001     |
|    loss                 | 0.0106     |
|    n_updates            | 5520       |
|    policy_gradient_loss | -0.0128    |
|    std                  | 0.124      |
|    value_loss           

Early stopping at step 11 due to reaching max kl: 0.15
policy iteration runtime: 24 seconds

Total episode rollouts: 145408

Eval num_timesteps=2560, episode_reward=0.85 +/- 0.00
Episode length: 5.00 +/- 0.00
----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 5          |
|    mean_reward          | 0.854      |
| time/                   |            |
|    fps                  | 353        |
|    iterations           | 1          |
|    time_elapsed         | 7          |
|    total_timesteps      | 2560       |
| train/                  |            |
|    approx_kl            | 0.15158339 |
|    clip_fraction        | 0.655      |
|    clip_range           | 0.15       |
|    entropy_loss         | 14.7       |
|    explained_variance   | 0.983      |
|    learning_rate        | 0.0001     |
|    loss                 | -0.0284    |
|    n_updates            | 5680       |
|    policy_gradient_loss | -0.00122   |
|    std    

Early stopping at step 13 due to reaching max kl: 0.16
policy iteration runtime: 28 seconds

Total episode rollouts: 149504

Eval num_timesteps=2560, episode_reward=0.85 +/- 0.00
Episode length: 5.00 +/- 0.00
--------------------------------------
| eval/                   |          |
|    mean_ep_length       | 5        |
|    mean_reward          | 0.854    |
| time/                   |          |
|    fps                  | 350      |
|    iterations           | 1        |
|    time_elapsed         | 7        |
|    total_timesteps      | 2560     |
| train/                  |          |
|    approx_kl            | 0.155241 |
|    clip_fraction        | 0.666    |
|    clip_range           | 0.15     |
|    entropy_loss         | 14.8     |
|    explained_variance   | 0.979    |
|    learning_rate        | 0.0001   |
|    loss                 | -0.03    |
|    n_updates            | 5840     |
|    policy_gradient_loss | -0.00598 |
|    std                  | 0.122    |
|    value_

<IPython.core.display.Javascript object>